In [1]:
from coverage_2 import CoverageEnv
from model import AdversarialModel
from trainer_2 import MultiPPOTrainer
from action_distribution import TorchHomogeneousMultiActionDistribution


In [2]:
policies = [
        '/Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-08_12-20-52/MultiPPOTrainer_coverage_aad49_00000_0_2023-03-08_12-20-52/checkpoint_000800',
        '/Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-05_22-30-26/MultiPPOTrainer_coverage_53dc4_00000_0_2023-03-05_22-30-27/checkpoint_000800',
        '/Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-06_18-53-08/MultiPPOTrainer_coverage_22bd0_00000_0_2023-03-06_18-53-08/checkpoint_000800'

    ]

In [11]:
import ray
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.algorithms.algorithm import Algorithm
from main_2 import CustomCallbacks

In [22]:
!pwd

/Users/antoniaboca/vs_code/R255/self_interested_comms


In [23]:
config={
            "callbacks": CustomCallbacks,
            "output": './eval_trace/',
            "framework": "torch",
            "env": "coverage",
            "use_gae": True,
            'use_critic': True,
            "kl_coeff": 0.5,
            "lambda": 0.95,
            "clip_param": 0.2,
            "entropy_coeff": 0.01,
            "train_batch_size": 1000,
            "rollout_fragment_length": 45,
            "sgd_minibatch_size": 100,
            "num_sgd_iter": 5,
            "num_gpus": 0,
            "num_workers": 1,
            "num_envs_per_worker": 5,
            "lr": 5e-4,
            "gamma": 0.9,
            "batch_mode": "truncate_episodes",
            "observation_filter": "NoFilter",
            "model": {
                "custom_model": "adversarial",
                "custom_action_dist": "hom_multi_action",
                "custom_model_config": {
                    'graph_layers': 1,
                    'graph_tabs': 2,
                    'graph_edge_features': 1,
                    'graph_features': 32,
                    'cnn_filters': [[8, [4, 4], 2], [16, [4, 4], 2], [32, [3, 3], 2]],
                    'value_cnn_filters': [[8, [4, 4], 2], [16, [4, 4], 2], [32, [4, 4], 2]],
                    'value_cnn_compression': 32,
                    'cnn_compression': 32,
                    'pre_gnn_mlp': [64, 128, 32],
                    'gp_kernel_size': 16,
                    'graph_aggregation': 'sum',
                    'relative': True,
                    'activation': 'relu',
                    'freeze_coop': False,
                    'freeze_greedy': False,
                    'freeze_coop_value': False,
                    'freeze_greedy_value': False,
                    'cnn_residual': False,
                    'agent_split': 1,
                    'greedy_mse_fac': 0.0,
                    'disable_comms': False,
                },
            },
            "env_config": {
                'world_shape': [16, 16],
                'state_size': 8,
                'collapse_state': False,
                'termination_no_new_coverage': 10,
                'max_episode_len': 154, # 16 * 16 * 0.6
                'n_agents': 4,
                'disabled_teams_step': [True, False],
                'disabled_teams_comms': [True, False],
                'min_coverable_area_fraction': 0.6,
                'map_mode': 'random',
                'reward_annealing': 0.0,
                'communication_range': 100.0,
                'ensure_connectivity': True,
                'reward_type': 'semi_cooperative', #semi_cooperative/cooperative
                'episode_termination': 'early', # early/fixed/default
                'operation_mode': 'coop_only',
            },
            'create_env_on_driver': True,
        }

In [15]:
ray.init()
register_env("coverage", lambda config: CoverageEnv(config))
ModelCatalog.register_custom_model("adversarial", AdversarialModel)
ModelCatalog.register_custom_action_dist(
    "hom_multi_action", TorchHomogeneousMultiActionDistribution
)

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [24]:
trainer = MultiPPOTrainer(config)

2023-03-09 17:56:29,281	WARNING ppo.py:351 -- `train_batch_size` (1000) cannot be achieved with your other settings (num_workers=1 num_envs_per_worker=5 rollout_fragment_length=45)! Auto-adjusting `rollout_fragment_length` to 200.
(RolloutWorker pid=56522) 2023-03-09 17:56:31,302	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=56522) 2023-03-09 17:56:31,302	WARNING env.py:159 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=56522) 2023-03-09 17:56:31,314	WARNING catalog.py:641 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them 

In [25]:
trainer.restore(policies[0])
trainer.evaluate({})

2023-03-09 17:56:39,037	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: /Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-08_12-20-52/MultiPPOTrainer_coverage_aad49_00000_0_2023-03-08_12-20-52/checkpoint_000800
2023-03-09 17:56:39,038	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 800, '_timesteps_total': None, '_time_total': 23060.995299339294, '_episodes_total': 245452}


{'evaluation': {'episode_reward_max': 69.0,
  'episode_reward_min': 0.0,
  'episode_reward_mean': 28.772519083969467,
  'episode_len_mean': 15.225954198473282,
  'episode_media': {},
  'episodes_this_iter': 655,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {'agent_reward_max_mean': 11.512977099236641,
   'agent_reward_max_min': 0,
   'agent_reward_max_max': 27,
   'agent_reward_min_mean': 3.1435114503816792,
   'agent_reward_min_min': 0,
   'agent_reward_min_max': 13,
   'agent_reward_mean_mean': 7.193129770992367,
   'agent_reward_mean_min': 0.0,
   'agent_reward_mean_max': 17.25},
  'hist_stats': {'episode_reward': [4.0,
    31.0,
    30.0,
    39.0,
    42.0,
    18.0,
    4.0,
    28.0,
    33.0,
    21.0,
    30.0,
    52.0,
    14.0,
    27.0,
    44.0,
    5.0,
    50.0,
    11.0,
    10.0,
    14.0,
    35.0,
    20.0,
    37.0,
    16.0,
    46.0,
    17.0,
    31.0,
    45.0,
    30.0,
    30.0,
    53.0,
    21.0,
    

In [27]:
trainer.restore(policies[1])
trainer.evaluate()

2023-03-09 17:58:25,528	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: /Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-05_22-30-26/MultiPPOTrainer_coverage_53dc4_00000_0_2023-03-05_22-30-27/checkpoint_000800
2023-03-09 17:58:25,528	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 800, '_timesteps_total': None, '_time_total': 22367.195441007614, '_episodes_total': 218612}
(raylet) [2023-03-09 17:58:37,684 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24716533760; capacity: 494384795648. Object creation will fail if spilling is required.


{'evaluation': {'episode_reward_max': 95.0,
  'episode_reward_min': 0.0,
  'episode_reward_mean': 37.69318181818182,
  'episode_len_mean': 18.97159090909091,
  'episode_media': {},
  'episodes_this_iter': 528,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {'agent_reward_max_mean': 13.861742424242424,
   'agent_reward_max_min': 0,
   'agent_reward_max_max': 30,
   'agent_reward_min_mean': 5.121212121212121,
   'agent_reward_min_min': 0,
   'agent_reward_min_max': 20,
   'agent_reward_mean_mean': 9.423295454545455,
   'agent_reward_mean_min': 0.0,
   'agent_reward_mean_max': 23.75},
  'hist_stats': {'episode_reward': [29.0,
    43.0,
    43.0,
    29.0,
    56.0,
    52.0,
    38.0,
    39.0,
    56.0,
    14.0,
    22.0,
    18.0,
    15.0,
    26.0,
    37.0,
    89.0,
    49.0,
    48.0,
    44.0,
    20.0,
    15.0,
    28.0,
    69.0,
    10.0,
    64.0,
    23.0,
    41.0,
    47.0,
    24.0,
    74.0,
    31.0,
    63.0,
    

(raylet) [2023-03-09 17:58:47,769 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24712228864; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 17:58:57,862 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24709812224; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 17:59:07,958 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24709689344; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 17:59:17,959 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24709570560; capacity: 494384795648. Object 

In [30]:
trainer.restore(policies[2])
trainer.evaluate()

2023-03-09 18:00:17,558	INFO trainable.py:766 -- Restored on 127.0.0.1 from checkpoint: /Users/antoniaboca/vs_code/R255/self_interested_comms/results/MultiPPOTrainer_2023-03-06_18-53-08/MultiPPOTrainer_coverage_22bd0_00000_0_2023-03-06_18-53-08/checkpoint_000800
2023-03-09 18:00:17,558	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 800, '_timesteps_total': None, '_time_total': 22019.655144929886, '_episodes_total': 219992}
(raylet) [2023-03-09 18:00:18,526 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24666963968; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 18:00:28,619 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24651382784; capacity: 494384795648. Object creation will fail if spilling is required.


{'evaluation': {'episode_reward_max': 103.0,
  'episode_reward_min': 0.0,
  'episode_reward_mean': 38.49227799227799,
  'episode_len_mean': 19.27799227799228,
  'episode_media': {},
  'episodes_this_iter': 518,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {'agent_reward_max_mean': 13.976833976833976,
   'agent_reward_max_min': 0,
   'agent_reward_max_max': 30,
   'agent_reward_min_mean': 5.428571428571429,
   'agent_reward_min_min': 0,
   'agent_reward_min_max': 23,
   'agent_reward_mean_mean': 9.623069498069498,
   'agent_reward_mean_min': 0.0,
   'agent_reward_mean_max': 25.75},
  'hist_stats': {'episode_reward': [11.0,
    14.0,
    56.0,
    60.0,
    53.0,
    67.0,
    43.0,
    11.0,
    37.0,
    38.0,
    27.0,
    38.0,
    48.0,
    28.0,
    54.0,
    14.0,
    64.0,
    61.0,
    13.0,
    18.0,
    21.0,
    54.0,
    62.0,
    20.0,
    48.0,
    9.0,
    8.0,
    76.0,
    23.0,
    16.0,
    60.0,
    1.0,
    27

(raylet) [2023-03-09 18:00:38,715 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24640856064; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 18:00:48,801 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24640851968; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 18:00:58,902 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24635711488; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-03-09 18:01:08,996 E 55026 1020386] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_16-37-44_243507_54877 is over 95% full, available space: 24635580416; capacity: 494384795648. Object 

In [ ]:
ray.shutdown()